In [ ]:
""" Quel périmetre?

- visite du site:
    - trustpilot filtre sur le pays (truspilot US, truspilot FR etc.)
    - catégories -> 22 (aliment, élétronique, santé etc.)
    - sous-catégories -> une dizaine max par catégories
        - pages catégorie -> nombre d'avis, score, le lieu, nom entreprise, sous-cat (+)
        - pages catégorie -> trier par avis pertinent, avec le plus d'avis, avis les plus recent
            - pages entreprise -> (+) titre de l'avis, le texte de l'avis, le lieu de l'avis, la date, le username, nombre davis effectué par la personne
            - pages entreprise -> réponse de l'entreprise, date
                - pages personne -> tous ces avis toute entreprise confondue

- on peut commencer par prendre le site Truspilote en francais -> les entreprises en france
- toute la france
- quelles catégories?
    - 
- problématique: 
    - quelles entreprises sont les mieux notées par catégories (banques, web etc.), les tendances par années
    - peut etre catégorie aliment + 
"""

In [ ]:
"""
Quelle solution pour récuperer de la données?:
- Scrapping
- API truspilot doc ici https://documentation-apidocumentation.trustpilot.com/
"""

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
#import pandas as pd

In [26]:
tpPage = urlopen("https://fr.trustpilot.com/categories/electronics_technology")
soup = BeautifulSoup(tpPage, 'html.parser')

tpTitles = soup.findAll(name = 'p', attrs = {'class': 'styles_displayName__1LIcI'})
tpMetrics = soup.findAll(name = 'p', attrs = {'class': 'styles_ratingText__nheL7'})
tpPlaces = soup.findAll(name = 'p', attrs = {'class': 'styles_location__3JATO'})


title = [] # On crée une liste vide qui contiendra tous les titres propres
for element in tpTitles:
    title.append(element.text)

metric = [] # On crée une liste vide qui contiendra tous les titres propres
for element in tpMetrics:
    metric.append(element.text)

place = [] # On crée une liste vide qui contiendra tous les titres propres
for element in tpPlaces:
    place.append(element.text)




In [28]:
print(place)

['Paris,\xa0France', 'Savignac-les-Eglises,\xa0France', 'Marseille,\xa0France', 'Rosières-près-Troyes,\xa0France', 'Lyon,\xa0France', 'Le Kremlin-Bicêtre,\xa0France', 'Lyon,\xa0France', 'Bayonne,\xa0France', 'Saumur,\xa0France', 'Bouaye,\xa0France', 'WITTERSDORF,\xa0France', 'Paris,\xa0France', 'La Palud sur Verdon,\xa0France', 'COLOMIERS,\xa0France', 'TAVERNY CEDEX,\xa0France', 'Paris,\xa0France', 'Strasbourg,\xa0France']
